In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv
import json
import pickle
import tqdm
#選択行削除
import re
from xml.sax.saxutils import unescape
from base64 import b64decode

import zipfile
import os
import itertools

from collections import OrderedDict
import traceback
from lxml.etree import fromstring as etree_fromstring

In [2]:
#自作関数読み込み
import pre_process.xbrl_process3 as xb

In [86]:
company = pd.read_csv("TDNet_full_data/company_list.csv",encoding="shift-jis",index_col=0)
#上場企業リスト取得
#証券コードのlistを作成
com_list=[]
for i in range(0,len(company)):
    if company["上場区分"][i]=="上場":
        com_list.append(company["証券コード"][i])
    #if len(com_list)==3882:
        #print(i)
#最初に書いてある，証券コードが記入されていない企業が最後に存在するため削除
com_list = com_list[:-1]

- 301 通期，連結，日本基準
- 304　1期，連結，日本基準
- 307 2期，連結，日本基準
- 310 3通期，連結，日本基準
- 313 その他四半期，連結，日本基準
- 350　通期の修正
- 352　2期の修正
- 354　その他期の修正
- 356　1期の修正
- 358 　3期の修正
- 360 配当予想の修正

In [7]:
def create_dic(dic):
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass

In [88]:
non_list=[]
no_pre_list=[]
pre_list=[]
df2_text=""
#for i in (range(0,10)):
for i in tqdm.tqdm(range(0,len(com_list))):
    
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for j in range(0,len(js["publiclyList"])):
        for k in range(0,len(js["publiclyList"][j]["disclosureItems"])):
            if (js["publiclyList"][j]["disclosureItems"][k][2:] == "301" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "304" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "307" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "310" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "313"):
                    if (js["publiclyList"][j]["xbrlFlag"]=="1") and (js["publiclyList"][j]["pdfSummaryFlag"]=="1"):
                        pdf_code_list.append(js["publiclyList"][j]["disclosureNumber"])
                        break

    for l in range(0,len(pdf_code_list)): 
        out='xbrl_data/{}/{}.htm'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_pkl='new2/non_pre/{}/{}.pkl'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_pkl2='new2/err_NoPre/{}/{}_NoPre.pkl'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_pkl3='new2/err_NoPre/{}/{}_table.pkl'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_pkl4='new2/err_pre/{}/{}_table.pkl'.format(js["publiclyList"][0]["code"],pdf_code_list[l])

        out_text='new2/xbrl_df/{}/{}.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_text3='new2/err_pre/{}/{}.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_text4='new2/err_NoPre/{}/{}.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])

        out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_csv2='new2/xbrl_df/{}/{}_2.csv'.format(js["publiclyList"][0]["code"],pdf_code_list[l])

        try:
            df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)
            dic1 = 'new2/xbrl_df/{}'.format(js["publiclyList"][0]["code"])
            dic2 = 'new2/non_pre/{}'.format(js["publiclyList"][0]["code"])
            dic3 = 'new2/err_NoPre/{}'.format(js["publiclyList"][0]["code"])
            dic4 = 'new2/err_pre/{}'.format(js["publiclyList"][0]["code"])

            #errflag,NoPreflagがfalseなら，データはあるのでファイルを作成
            if not errFlag and not NoPreFlag:
                #フォルダを作成する
                create_dic(dic1)
                df1.to_csv(out_csv1)
                df2.to_csv(out_csv2)
                with open(out_text,'w') as out:
                    out.write(pre_term)

              #errflag=false,NoPreflag=Trueなら，予測なし
            elif not errFlag and NoPreFlag:
                #フォルダを作成する
                create_dic(dic1)
                df1.to_csv(out_csv1)
                with open(out_text,'w') as out:
                    out.write(pre_term) 
                with open(out_txt2,'w') as out:
                    df2_text="No Prediction"
                    out.write(df2_text)

                #フォルダを作成する
                create_dic(dic2)
                #token保
                with open(out_pkl, 'wb') as p:
                    pickle.dump(NoPredict_text, p)

            #errにはなったが，予測がないことはわかっている場合
            elif errFlag and  NoPreFlag:

                no_pre_list.append(out)
                 #フォルダを作成する
                create_dic(dic3)
                with open(out_pkl2, 'wb') as p:
                    pickle.dump(NoPredict_text, p)
                with open(out_pkl3, 'wb') as p:
                    pickle.dump(table_list, p)
                with open(out_text4,'w') as out:
                    out.write(pre_term) 

            #err        
            else:
                pre_list.append(out)
                create_dic(dic4)
                with open(out_pkl4, 'wb') as p:
                    pickle.dump(table_list, p)
                with open(out_text3,'w') as out:
                    out.write(pre_term) 
                    
        except:
            out='xbrl_data/{}/{}.htm'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
            non_list.append(out)

100%|███████████████████████████████████████| 3881/3881 [28:07<00:00,  2.30it/s]


In [89]:
print(len(no_pre_list))
print(len(pre_list))
print(len(non_list))

3272
911
0


In [90]:
no_pre_list

['xbrl_data/13330/20200731471206.htm',
 'xbrl_data/13330/20200513412695.htm',
 'xbrl_data/57110/20200427400111.htm',
 'xbrl_data/57140/20200709459868.htm',
 'xbrl_data/57140/20200525423896.htm',
 'xbrl_data/57150/20200426499717.htm',
 'xbrl_data/96750/20210810482886.htm',
 'xbrl_data/96750/20210512412578.htm',
 'xbrl_data/96750/20200731471024.htm',
 'xbrl_data/96750/20200513411516.htm',
 'xbrl_data/70210/20200428401753.htm',
 'xbrl_data/15180/20200511408829.htm',
 'xbrl_data/18030/20200522422202.htm',
 'xbrl_data/18020/20200318481544.htm',
 'xbrl_data/18150/20200424499542.htm',
 'xbrl_data/18110/20200501404113.htm',
 'xbrl_data/19410/20200416495242.htm',
 'xbrl_data/19420/20200428401321.htm',
 'xbrl_data/19440/20200413493391.htm',
 'xbrl_data/19460/20200422497260.htm',
 'xbrl_data/18920/20200514413667.htm',
 'xbrl_data/18850/20200507405980.htm',
 'xbrl_data/18130/20200512410886.htm',
 'xbrl_data/18410/20200508407240.htm',
 'xbrl_data/18350/20200416495146.htm',
 'xbrl_data/19680/2020042

In [91]:
pre_list

['xbrl_data/14910/20200513412376.htm',
 'xbrl_data/15150/20200429402145.htm',
 'xbrl_data/19450/20210427401134.htm',
 'xbrl_data/19640/20200430402931.htm',
 'xbrl_data/18480/20210201453300.htm',
 'xbrl_data/22160/20180801490075.htm',
 'xbrl_data/22160/20180508430012.htm',
 'xbrl_data/28010/20210421497149.htm',
 'xbrl_data/29020/20200729468827.htm',
 'xbrl_data/29020/20200427400218.htm',
 'xbrl_data/31130/20220511540034.htm',
 'xbrl_data/31130/20220202578035.htm',
 'xbrl_data/31130/20211102423049.htm',
 'xbrl_data/31130/20210804478749.htm',
 'xbrl_data/31130/20210512412686.htm',
 'xbrl_data/31130/20210203454923.htm',
 'xbrl_data/31130/20201028410607.htm',
 'xbrl_data/31130/20200730470447.htm',
 'xbrl_data/31130/20200512410238.htm',
 'xbrl_data/31130/20200205457900.htm',
 'xbrl_data/31130/20191106420180.htm',
 'xbrl_data/31130/20190801480391.htm',
 'xbrl_data/31130/20190507415713.htm',
 'xbrl_data/31130/20190205470112.htm',
 'xbrl_data/31130/20181107431874.htm',
 'xbrl_data/31130/2018080

In [92]:
non_list

[]

In [93]:
dic1={}
for i in range(0,len(no_pre_list)):
    if no_pre_list[i][10:15] in dic1.keys():
        dic1[no_pre_list[i][10:15]]+=1
    else:
        dic1[no_pre_list[i][10:15]]=1

In [94]:
len(dic1.keys())

1275

In [95]:
cnt=0
for i in dic1.values():
    if i<3:
        cnt+=1
print(cnt)

997


In [96]:
1275-997

278

In [97]:
dic1

{'13330': 2,
 '57110': 1,
 '57140': 2,
 '57150': 1,
 '96750': 4,
 '70210': 1,
 '15180': 1,
 '18030': 1,
 '18020': 1,
 '18150': 1,
 '18110': 1,
 '19410': 1,
 '19420': 1,
 '19440': 1,
 '19460': 1,
 '18920': 1,
 '18850': 1,
 '18130': 1,
 '18410': 1,
 '18350': 1,
 '19680': 1,
 '19490': 1,
 '19500': 2,
 '18500': 1,
 '18520': 1,
 '19340': 2,
 '19670': 1,
 '19590': 1,
 '19690': 1,
 '19750': 1,
 '19600': 1,
 '19390': 1,
 '19790': 1,
 '19380': 2,
 '19800': 1,
 '19660': 3,
 '19730': 1,
 '18780': 1,
 '18790': 2,
 '19650': 1,
 '19520': 2,
 '19050': 1,
 '19040': 1,
 '18400': 1,
 '17980': 1,
 '17830': 2,
 '17750': 1,
 '17700': 2,
 '17660': 2,
 '17620': 1,
 '17570': 10,
 '17360': 1,
 '17200': 1,
 '17190': 1,
 '21070': 1,
 '22220': 5,
 '25990': 1,
 '28010': 2,
 '42290': 2,
 '80060': 2,
 '29030': 1,
 '29050': 3,
 '28130': 4,
 '29070': 1,
 '29100': 2,
 '28150': 1,
 '25940': 4,
 '29150': 2,
 '28770': 2,
 '25970': 8,
 '28310': 2,
 '29230': 4,
 '29240': 2,
 '28180': 2,
 '30020': 1,
 '30100': 10,
 '30010': 

In [98]:
dic2={}
for i in range(0,len(pre_list)):
    if pre_list[i][10:15] in dic2.keys():
        dic2[pre_list[i][10:15]]+=1
    else:
        dic2[pre_list[i][10:15]]=1

In [99]:
len(dic2.keys())

222

In [100]:
cnt=0
for i in dic2.values():
    if i<3:
        cnt+=1
print(cnt)

135


In [101]:
222-135

87

In [103]:
#token保存
out='new2/NoPre/non_list.pkl'
with open(out, 'wb') as p:
    pickle.dump(no_pre_list, p)

# errの処理

In [4]:
#token保存
import pickle
out='new2/NoPre/non_list.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [5]:
out_list[0][10:15]
out_list[0][16:-4]
#out_list[0]

'20200731471206'

In [6]:
len(out_list)

3272

In [26]:
l=0
out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)

In [27]:
NoPreFlag

True

In [20]:
non_list=[]
for l in tqdm.tqdm(range(0,len(out_list))): 
    out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])

    out_text='new2/xbrl_df/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])

    out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    try:
        df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)
        dic1 = 'new2/xbrl_df/{}'.format(out_list[l][10:15],out_list[l][16:-4])
        dic2 = 'new2/non_pre/{}'.format(js["publiclyList"][0]["code"])
        
        
        #errflag=false,NoPreflag=Trueなら，予測なし
        if NoPreFlag:
        #フォルダを作成する
            create_dic(dic1)
            df1.to_csv(out_csv1)
            with open(out_text,'w') as out:
                out.write(pre_term) 
            with open(out_txt2,'w') as out:
                df2_text="No Prediction"
                out.write(df2_text)
                
            dic2 = 'new2/non_pre/{}'.format(js["publiclyList"][0]["code"])
            
        else:
            out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
            non_list.append(out)
            
    except:
        out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
        non_list.append(out)

100%|███████████████████████████████████████| 3272/3272 [01:29<00:00, 36.72it/s]


In [21]:
len(non_list)

0

In [22]:
non_list

[]

# err２つ目

In [3]:
#自作関数読み込み
import pre_process.xbrl_process4 as xb

In [14]:
#token保存
import pickle
out='new2/err/non_list.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [15]:
len(out_list)

911

In [25]:
xb.main(out_list[l])

(              0      1     2     3     4     5                6  \
 None        百万円      ％   百万円     ％   百万円     ％              百万円   
 2022年3月期  12577    9.8  -174     ―   -62     ―              -85   
 2021年3月期  11458  -14.6  -627     ―  -491     ―             -682   
 index       売上高    売上高  営業利益  営業利益  経常利益  経常利益  親会社株主に帰属する当期純利益   
 
                         7           8                  9  
 None                    ％          円銭                 円銭  
 2022年3月期                ―       -4.89                  ―  
 2021年3月期                ―      -24.11                  ―  
 index     親会社株主に帰属する当期純利益  1株当たり当期純利益  潜在株式調整後1株当たり当期純利益  ,
 None,
 '2023年3月期の連結業績予想',
 ['\n当社は、「企業は社会の公器」として、短期的な利益ではなく、長期的な視点に立った経営を優先する社会の実現を目指す、そのような企業群の一翼を担うことが重要な経営目標であると認識しています。したがって、社員・顧客・仕入先・地域社会・地球といったすべての社中に貢献することにより企業価値を上げ、その結果として株主にも利益をもたらすという考え方である「公益資本主義」の理念に賛同しています。None四半期決算の開示あるいは業績予想の開示については、当該諸制度・諸規則(慣習)のありかたそのものによって、目先の株価変動や短期業績への過剰な傾注、投資意思決定における投機的な視野への偏狭を誘発し、企業活動における長期的な視点に立った研究開発投資、設備投資および人材

In [16]:
non_list=[]
for l in tqdm.tqdm(range(0,len(out_list))): 
    out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])

    out_text='new2/xbrl_df/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt3='new2/NoPre/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])

    
    out_pkl='new2/NoPre/{}/{}.pkl'.format(out_list[l][10:15],out_list[l][16:-4])
    
    out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    out_csv2='new2/xbrl_df/{}/{}_2.csv'.format(out_list[l][10:15],out_list[l][16:-4])

    df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)
    dic1 = 'new2/xbrl_df/{}'.format(out_list[l][10:15],out_list[l][16:-4])
    dic2 = 'new2/NoPre/{}'.format(out_list[l][10:15],out_list[l][16:-4])

    #errflag,NoPreflagがfalseなら，データはあるのでファイルを作成
    if not errFlag and not NoPreFlag:
        #フォルダを作成する
        create_dic(dic1)
        df1.to_csv(out_csv1)
        df2.to_csv(out_csv2)
        with open(out_text,'w') as out:
            out.write(pre_term)

    #errflag=false,NoPreflag=Trueなら，予測なし
    elif NoPreFlag:
    #フォルダを作成する
        create_dic(dic1)
        df1.to_csv(out_csv1)
        with open(out_text,'w') as out:
            out.write(pre_term) 
        with open(out_txt2,'w') as out:
            df2_text="No Prediction"
            out.write(df2_text)

        create_dic(dic2)
        #token保
        with open(out_pkl, 'wb') as p:
            pickle.dump(NoPredict_text, p)

        with open(out_txt3,'w') as out:
            out.write(pre_term) 

    else:
        out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
        non_list.append(out)

100%|█████████████████████████████████████████| 911/911 [00:23<00:00, 38.51it/s]


In [17]:
len(non_list)

146

In [10]:
non_list

['xbrl_data/34070/20190510420533.htm',
 'xbrl_data/53430/20220510537296.htm',
 'xbrl_data/53430/20210324483627.htm',
 'xbrl_data/53430/20200416494750.htm',
 'xbrl_data/53430/20190507415202.htm',
 'xbrl_data/53430/20190123462529.htm',
 'xbrl_data/53430/20181002413621.htm',
 'xbrl_data/53430/20180726485386.htm',
 'xbrl_data/53430/20180302481598.htm',
 'xbrl_data/54010/20190205470139.htm',
 'xbrl_data/54010/20181030425598.htm',
 'xbrl_data/54010/20180706477808.htm',
 'xbrl_data/59330/20220428531754.htm',
 'xbrl_data/59330/20220131576772.htm',
 'xbrl_data/59330/20211019413502.htm',
 'xbrl_data/59330/20210720469325.htm',
 'xbrl_data/59330/20210426499613.htm',
 'xbrl_data/59330/20210129451596.htm',
 'xbrl_data/61430/20171114417968.htm',
 'xbrl_data/61430/20170809453320.htm',
 'xbrl_data/67150/20190614455003.htm',
 'xbrl_data/67150/20190419407282.htm',
 'xbrl_data/72010/20171106410086.htm',
 'xbrl_data/79450/20220510537341.htm',
 'xbrl_data/81070/20171108412761.htm',
 'xbrl_data/81540/2018050

In [18]:
dic1={}
for i in range(0,len(non_list)):
    if non_list[i][10:15] in dic1.keys():
        dic1[non_list[i][10:15]]+=1
    else:
        dic1[non_list[i][10:15]]=1

In [19]:
len(dic1.keys())

45

In [20]:
#token保存
out='new2/err/non_list2.pkl'
with open(out, 'wb') as p:
    pickle.dump(non_list, p)

# 3つめ

In [22]:
#token保存
import pickle
out='new2/err/non_list2.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [21]:
#目視で確認したもの
pre_dic={
    78650:1,
    48000:1,
    37930:1,
    30420:1,
    67400:1,
    39030:1,
    34070:1,
    59330:1,
    83030:1,
    23890:1,
    36880:1,
    94680:1,
    31330:1,
    39940:1,
    29820:1,
    73390:1
}

In [43]:
non_pre_list[0]

'xbrl_data/53430/20220510537296.htm'

In [42]:
pre_list

['xbrl_data/34070/20190510420533.htm',
 'xbrl_data/59330/20220428531754.htm',
 'xbrl_data/59330/20220131576772.htm',
 'xbrl_data/59330/20211019413502.htm',
 'xbrl_data/59330/20210720469325.htm',
 'xbrl_data/59330/20210426499613.htm',
 'xbrl_data/59330/20210129451596.htm',
 'xbrl_data/78650/20210301471772.htm',
 'xbrl_data/78650/20200214465867.htm',
 'xbrl_data/78650/20181218451312.htm',
 'xbrl_data/78650/20180228480150.htm',
 'xbrl_data/83030/20180104445808.htm',
 'xbrl_data/83030/20170928480493.htm',
 'xbrl_data/48000/20190502414268.htm',
 'xbrl_data/23890/20190206471708.htm',
 'xbrl_data/23890/20190207472510.htm',
 'xbrl_data/23890/20190207472614.htm',
 'xbrl_data/23890/20181108433037.htm',
 'xbrl_data/23890/20180802491018.htm',
 'xbrl_data/23890/20180509431497.htm',
 'xbrl_data/37930/20210406490409.htm',
 'xbrl_data/37930/20200506404856.htm',
 'xbrl_data/37930/20190430414056.htm',
 'xbrl_data/37930/20180502426993.htm',
 'xbrl_data/30420/20220513547595.htm',
 'xbrl_data/30420/2021051

In [41]:
non_list=[]
non_pre_list=[]
pre_list=[]
for l in tqdm.tqdm(range(0,len(out_list))): 
    if int(out_list[l][10:15]) in pre_dic.keys():
        pre_list.append(out_list[l])
    else:
        non_pre_list.append(out_list[l])
        out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])

        out_text='new2/xbrl_df/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
        out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
        out_txt3='new2/NoPre/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])


        out_pkl='new2/NoPre/{}/{}.pkl'.format(out_list[l][10:15],out_list[l][16:-4])

        out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
        try:
            df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)
        except:
            break

100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 69.61it/s]


In [45]:
non_list=[]
pre_list=[]
for l in tqdm.tqdm(range(0,len(out_list))): 

    if int(out_list[l][10:15]) in pre_dic.keys():
        pre_list.append(out_list[l])
    else:
        out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])

        out_text='new2/xbrl_df/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
        out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
        out_txt3='new2/NoPre/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])


        out_pkl='new2/NoPre/{}/{}.pkl'.format(out_list[l][10:15],out_list[l][16:-4])

        out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
        try:
            df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)
            dic1 = 'new2/xbrl_df/{}'.format(out_list[l][10:15],out_list[l][16:-4])
            dic2 = 'new2/NoPre/{}'.format(out_list[l][10:15],out_list[l][16:-4])
            #フォルダを作成する
            create_dic(dic1)
            df1.to_csv(out_csv1)
            with open(out_text,'w') as out:
                out.write(pre_term) 
            with open(out_txt2,'w') as out:
                df2_text="No Prediction"
                out.write(df2_text)

            create_dic(dic2)
            #token保
            with open(out_pkl, 'wb') as p:
                pickle.dump(NoPredict_text, p)

            with open(out_txt3,'w') as out:
                out.write(pre_term) 
        except:
            out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
            non_list.append(out)

100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 66.07it/s]


In [46]:
import os

path = 'new2/NoPre'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

1346


In [47]:
#token保存
out='new2/err/non_list3.pkl'
with open(out, 'wb') as p:
    pickle.dump(pre_list, p)

In [48]:
out='new2/err/non_list3.pkl'
with open(out, 'rb') as p:
    t=pickle.load(p)

In [49]:
t

['xbrl_data/34070/20190510420533.htm',
 'xbrl_data/59330/20220428531754.htm',
 'xbrl_data/59330/20220131576772.htm',
 'xbrl_data/59330/20211019413502.htm',
 'xbrl_data/59330/20210720469325.htm',
 'xbrl_data/59330/20210426499613.htm',
 'xbrl_data/59330/20210129451596.htm',
 'xbrl_data/78650/20210301471772.htm',
 'xbrl_data/78650/20200214465867.htm',
 'xbrl_data/78650/20181218451312.htm',
 'xbrl_data/78650/20180228480150.htm',
 'xbrl_data/83030/20180104445808.htm',
 'xbrl_data/83030/20170928480493.htm',
 'xbrl_data/48000/20190502414268.htm',
 'xbrl_data/23890/20190206471708.htm',
 'xbrl_data/23890/20190207472510.htm',
 'xbrl_data/23890/20190207472614.htm',
 'xbrl_data/23890/20181108433037.htm',
 'xbrl_data/23890/20180802491018.htm',
 'xbrl_data/23890/20180509431497.htm',
 'xbrl_data/37930/20210406490409.htm',
 'xbrl_data/37930/20200506404856.htm',
 'xbrl_data/37930/20190430414056.htm',
 'xbrl_data/37930/20180502426993.htm',
 'xbrl_data/30420/20220513547595.htm',
 'xbrl_data/30420/2021051

# 予測を取得しないようにする

In [7]:
df1_none_list=[]
df2_none_list=[]
#for i in (range(100,110)):
for i in tqdm.tqdm(range(0,len(com_list))):
    
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for j in range(0,len(js["publiclyList"])):
        for k in range(0,len(js["publiclyList"][j]["disclosureItems"])):
            if (js["publiclyList"][j]["disclosureItems"][k][2:] == "301" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "304" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "307" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "310" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "313"):
                    if (js["publiclyList"][j]["xbrlFlag"]=="1") and (js["publiclyList"][j]["pdfSummaryFlag"]=="1"):
                        pdf_code_list.append(js["publiclyList"][j]["disclosureNumber"])
                        break

    for l in range(0,len(pdf_code_list)): 
        out_htm='xbrl_data/{}/{}.htm'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_text='xbrl_df/{}/{}.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_csv1='xbrl_df/{}/{}_1.csv'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_csv2='xbrl_df/{}/{}_2.csv'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        out_txt2='xbrl_df/{}/{}_err.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[l])
        dic = 'xbrl_df/{}'.format(js["publiclyList"][0]["code"])
        #フォルダを作成する
        try:
            os.makedirs(dic)
        except FileExistsError:
            pass
        
        try:
            df1,df2,pre_term=xb.main(out_htm)
            df1.to_csv(out_csv1)
            with open(out_text,'w') as out:
                out.write(pre_term)
                
            if type(df2)==str:
                df2_none_list.append(out_htm)
                with open(out_txt2,'w') as out:
                    out.write(df2)
            else:
                df2.to_csv(out_csv2)
        except:
            df1_none_list.append(out_htm)

100%|███████████████████████████████████████| 3881/3881 [27:56<00:00,  2.31it/s]


In [10]:
len(df2_none_list)

1154

In [11]:
len(df1_none_list)

213

In [12]:
df2_none_list

['xbrl_data/16050/20190808485390.htm',
 'xbrl_data/18020/20200318481544.htm',
 'xbrl_data/19540/20201214434429.htm',
 'xbrl_data/19540/20200831487445.htm',
 'xbrl_data/19540/20191113425661.htm',
 'xbrl_data/19540/20190830493198.htm',
 'xbrl_data/19540/20190813487723.htm',
 'xbrl_data/19540/20181112435295.htm',
 'xbrl_data/19540/20180813497206.htm',
 'xbrl_data/19540/20171113416849.htm',
 'xbrl_data/19540/20170814456108.htm',
 'xbrl_data/81570/20210730474968.htm',
 'xbrl_data/97670/20220209585289.htm',
 'xbrl_data/22160/20180801490075.htm',
 'xbrl_data/22160/20180508430012.htm',
 'xbrl_data/22160/20180206464984.htm',
 'xbrl_data/25930/20220530562658.htm',
 'xbrl_data/26070/20210511410760.htm',
 'xbrl_data/26070/20200512410153.htm',
 'xbrl_data/28010/20210421497149.htm',
 'xbrl_data/28010/20200721465625.htm',
 'xbrl_data/28010/20200424498630.htm',
 'xbrl_data/29050/20220420524807.htm',
 'xbrl_data/28310/20180514437179.htm',
 'xbrl_data/35280/20200213463264.htm',
 'xbrl_data/35280/2019111

In [13]:
df1_none_list

['xbrl_data/81570/20220513546415.htm',
 'xbrl_data/81570/20210514419124.htm',
 'xbrl_data/21090/20211029421013.htm',
 'xbrl_data/21090/20210806481408.htm',
 'xbrl_data/21090/20201029411629.htm',
 'xbrl_data/21090/20200730470784.htm',
 'xbrl_data/21090/20191031417090.htm',
 'xbrl_data/21090/20190730478714.htm',
 'xbrl_data/21090/20181030425439.htm',
 'xbrl_data/21090/20180731488821.htm',
 'xbrl_data/21090/20171030403576.htm',
 'xbrl_data/21090/20170726439531.htm',
 'xbrl_data/26020/20210805480114.htm',
 'xbrl_data/26020/20200805475540.htm',
 'xbrl_data/26020/20190801481047.htm',
 'xbrl_data/43680/20220509535456.htm',
 'xbrl_data/43680/20210507407625.htm',
 'xbrl_data/43680/20200508407104.htm',
 'xbrl_data/58050/20220203579175.htm',
 'xbrl_data/58050/20210730475691.htm',
 'xbrl_data/58050/20210202453707.htm',
 'xbrl_data/58050/20200728468057.htm',
 'xbrl_data/58050/20200130454014.htm',
 'xbrl_data/58050/20190724475309.htm',
 'xbrl_data/58050/20190122462011.htm',
 'xbrl_data/58050/2018072

In [19]:
df2_none_list[0][10:15]

'16050'

In [26]:
dic2={}
for i in range(0,len(df2_none_list)):
    dic2[df2_none_list[i][10:15]]=1

In [27]:
dic1={}
for i in range(0,len(df1_none_list)):
    dic1[df1_none_list[i][10:15]]=1

In [31]:
len(dic1.keys())

36

In [32]:
len(dic2.keys())

290

In [20]:
out='new/non_pre/13330/20200731471206.pkl'
with open(out, 'rb') as p:
    a=pickle.load(p)

In [21]:
a

['',
 '(注)直近に公表されている業績予想からの修正の有無有None2021年3月期第2四半期(累計)の連結業績予想の修正につきましては、本日(2020年8月3日)公表いたしました「業績予想の修正に関するお知らせ」をご覧ください。Noneなお、2021年3月期の通期連結業績予想につきましては、新型コロナウイルス感染拡大について予断を許さない状況であり、業績予想が困難なことから、今後、適切かつ合理的な予想が算定可能となった時点で速やかに公表いたします。None※None注記事項(1)当四半期連結累計期間における重要な子会社の異動(連結範囲の変更を伴う特定子会社の異動)無None(2)四半期連結財務諸表の作成に特有の会計処理の適用有(注)詳細は、〔添付資料〕Ｐ.6「2.四半期連結財務諸表及び主な注記(3)四半期連結財務諸表に関する注記事項(四半期連結財務諸表の作成に特有の会計処理の適用)」をご覧ください。None(3)会計方針の変更・会計上の見積りの変更・修正再表示①None会計基準等の改正に伴う会計方針の変更NoneNoneNoneNoneNoneNoneNone無②None①以外の会計方針の変更NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone無③None会計上の見積りの変更NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone無④None修正再表示NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone無None(4)発行済株式数(普通株式)(注)直近に公表されている業績予想からの修正の有無有None2021年3月期第2四半期(累計)の連結業績予想の修正につきましては、本日(2020年8月3日)公表いたしました「業績予想の修正に関するお知らせ」をご覧ください。Noneなお、2021年3月期の通期連結業績予想につ

In [25]:
for i in (range(0,10)):
#for i in tqdm.tqdm(range(0,len(com_list))):
    
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for j in range(0,len(js["publiclyList"])):
        for k in range(0,len(js["publiclyList"][j]["disclosureItems"])):
            if (js["publiclyList"][j]["disclosureItems"][k][2:] == "301" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "304" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "307" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "310" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "313"):
                    if (js["publiclyList"][j]["xbrlFlag"]=="1") and (js["publiclyList"][j]["pdfSummaryFlag"]=="1"):
                        pdf_code_list.append(js["publiclyList"][j]["disclosureNumber"])
                        break
    print(pdf_code_list)

['20220328512394', '20211224560318', '20210928403538', '20210708463658', '20210331488354', '20201224438982', '20201001498500', '20200713461156', '20200327485606', '20191219439108', '20190919499244', '20190624460038', '20190307487203', '20181205445580', '20180905403871', '20180613463090', '20180301480984', '20171201428656', '20170901466541', '20170615407199']
['20220329513011', '20220107564830', '20211012409694', '20210712464804', '20210407491046', '20210108442531', '20201008401253', '20200710461009', '20200401488480', '20200108444560', '20191003404524', '20190709469566', '20190404400399', '20190111458900', '20181004414611', '20180711479831', '20180402403454', '20180110448503', '20171005486032', '20170703423898']
[]
['20220513547222', '20220210586586', '20211112434542', '20210812484788', '20210514419685', '20210202453889', '20201102413786', '20200803472992', '20200515416026', '20200203455814', '20191105419115', '20190802481399', '20190510421358', '20190204469432', '20181102428743', '201

In [24]:
pdf_code_list

['20220506534334',
 '20220203579412',
 '20211104424989',
 '20210805479869',
 '20210507408047',
 '20210205456672',
 '20201029411734',
 '20200731471206',
 '20200513412695',
 '20200131455025',
 '20191031416661',
 '20190801481023',
 '20190509418279',
 '20190204469032',
 '20181102428838',
 '20180802490377',
 '20180501426212',
 '20180201461339',
 '20171101405770',
 '20170804448369']

In [25]:
import os

path = 'new/xbrl_df'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

2074


In [29]:
path = 'new/non_pre'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

723


In [26]:
path = 'new/err_pre'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

1817


In [28]:
path = 'new/err_NoPre'
folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

579


In [30]:
723+579

1302

In [32]:
2074+1817

3891

# 予測ないものの確認

In [8]:
for l in tqdm.tqdm(range(0,len(out_list))): 
    out='new2/NoPre/{}/{}_NoPre.pkl'.format(out_list[l][10:15],out_list[l][16:-4])
    break

  0%|                                                  | 0/3272 [00:00<?, ?it/s]


In [17]:
l=6
out='new2/NoPre/{}/{}_NoPre.pkl'.format(out_list[l][10:15],out_list[l][16:-4])
with open(out, 'rb') as p:
    a=pickle.load(p)
a

['\n3.2022年3月期の連結業績予想(2021年4月1日～2022年3月31日)2022年3月期の連結業績予想につきましては、現時点では合理的に算定することが困難であるため、未定といたします。今後、連結業績予想の算定が可能となった時点で速やかに開示いたします。(注)直近に公表されている業績予想からの修正の有無無trend\n',
 '(注)当第1四半期連結会計期間の期首より、「収益認識に関する会計基準」(企業会計基準第29号2020年3月31日)等を適用しております。従いまして、前第1四半期連結累計期間との会計処理が異なることから売上高の増減率については記載しておりません。(2)連結財政状態(注)当第1四半期連結会計期間の期首より、「収益認識に関する会計基準」(企業会計基準第29号2020年3月31日)等を適用しております。従いまして、前第1四半期連結累計期間との会計処理が異なることから売上高の増減率については記載しておりません。(2)連結財政状態table\n総資産\n純資産\n自己資本比率trend\n百万円\n百万円\n％trend\ntrend\n2022年3月期第1四半期\n52651\n9699\n18.4trend\n2021年3月期\n50297\n10853\n21.5trendtable\n3.2022年3月期の連結業績予想(2021年4月1日～2022年3月31日)2022年3月期の連結業績予想につきましては、現時点では合理的に算定することが困難であるため、未定といたします。今後、連結業績予想の算定が可能となった時点で速やかに開示いたします。(注)直近に公表されている業績予想からの修正の有無無trend']

# errの確認

In [1]:
#token保存
import pickle
out='new2/err/non_list.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [2]:
for l in range(0,len(out_list)):
    out='new2/err/{}/{}_table.pkl'.format(out_list[l][10:15],out_list[l][16:-4])
    with open(out, 'rb') as p:
        a=pickle.load(p)
    print(a)
    print("*****************************************************************************")

['\n売上高\n営業利益\n経常利益\n親会社株主に帰属する当期純利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2020年3月期\n28517\n7.8\n-813\n―\n-893\n―\n-296\n―trend\n2019年3月期\n26457\n-49.2\n593\n―\n532\n―\n123\n―trend\n', '\n1株当たり当期純利益\n潜在株式調整後1株当たり当期純利益\n自己資本当期純利益率\n総資産経常利益率\n売上高営業利益率trend\n円銭\n円銭\n％\n％\n％trend\n2020年3月期\n-1.03\n―\n-4.5\n-11.5\n-2.9trend\n2019年3月期\n0.43\n―\n1.9\n7.1\n2.2trend\n', '\n売上高\n営業利益\n経常利益\n当期純利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2020年3月期\n28791\n18.1\n351\n―\n259\n―\n232\n―trend\n2019年3月期\n24382\n-52.6\n-251\n―\n-275\n―\n-284\n―trend\n', '\n1株当たり当期純利益\n潜在株式調整後1株当たり当期純利益trend\n円銭\n円銭trend\n2020年3月期\n0.80\n―trend\n2019年3月期\n-0.98\n―trend\n']
*****************************************************************************
['\n売上高\n営業利益\n経常利益\n親会社株主に帰属する当期純利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2年3月期\n117502\n-4.8\n7576\n1.3\n8012\n8.9\n4518\n-15.7trend\n31年3月期\n123372\n3.9\n7479\n-11.7\n7356\n-16.2\n5360\n9.9trend\n', '\n1株当たり当期純利益\n潜在株式調整後1株当たり当期純利益\n自己資本当期純利益率\n総資産経常利益率\n売上高営業利

In [28]:
out

'new2/err/19640/20200430402931_table.pkl'

In [ ]:
non_list=[]
for l in tqdm.tqdm(range(0,len(out_list))): 
    out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])

    out_text='new2/xbrl_df/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt2='new2/xbrl_df/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])

    out_csv1='new2/xbrl_df/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list=xb.main(out)

In [36]:
l=0
out='xbrl_data/{}/{}.htm'.format(out_list[l][10:15],out_list[l][16:-4])
xb.main(out)

(              0      1     2     3     4     5                6  \
 None        百万円      ％   百万円     ％   百万円     ％              百万円   
 2020年3月期  28517    7.8  -813     ―  -893     ―             -296   
 2019年3月期  26457  -49.2   593     ―   532     ―              123   
 index       売上高    売上高  営業利益  営業利益  経常利益  経常利益  親会社株主に帰属する当期純利益   
 
                         7           0                  1  
 None                    ％          円銭                 円銭  
 2020年3月期                ―        0.80                  ―  
 2019年3月期                ―       -0.98                  ―  
 index     親会社株主に帰属する当期純利益  1株当たり当期純利益  潜在株式調整後1株当たり当期純利益  ,
 None,
 '2021年3月期の連結業績予想',
 ['\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等については、添付資料Ｐ.3「1.経営成績等の概況(4)今後の見通し」をご覧ください。trend\n',
  'table\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項本

In [37]:
out

'xbrl_data/14910/20200513412376.htm'